In [15]:
import numpy as np 
import pandas as pd
import json

import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
meta1 = pd.read_csv('user.csv')
meta1.head()

#아이템기반필터링#

,userid,rating,contentId
0,1,10,a62
1,2,2,a34
2,3,8,a52
3,4,6,a63
4,5,5,a44


In [17]:
meta = pd.read_csv('teaable.csv')

meta.head()

,차품목,효능번호,카페인,향기준,contentId,"맛,향",personId
0,Bengal Spice,1,X,2,a1,"계피, 생강",1
1,BerryBlossom White,1,O,1,a2,과일향(베리향),2
2,Black Cherry Berry,3,X,1,a3,"과일향(체리, 베리)",3
3,Breakfast in Paris Black Tea,1,O,"1,4",a4,"라벤더꽃향, 시트러스향",4
4,ButterScotch Blondie,3,O,4,a5,"버터카라멜향, 바닐라향",5


In [18]:
ratings_tea = pd.merge(meta1, meta, on = 'contentId')

In [19]:
df = pd.DataFrame(ratings_tea)
df.head(10)

,userid,rating,contentId,차품목,효능번호,카페인,향기준,"맛,향",personId
0,1,10,a62,ZEN™-green tea,3,O,3,시원한 향,62
1,21,4,a62,ZEN™-green tea,3,O,3,시원한 향,62
2,57,3,a62,ZEN™-green tea,3,O,3,시원한 향,62
3,61,1,a62,ZEN™-green tea,3,O,3,시원한 향,62
4,78,8,a62,ZEN™-green tea,3,O,3,시원한 향,62
5,85,3,a62,ZEN™-green tea,3,O,3,시원한 향,62
6,2,2,a34,Lemon Mint,4,X,1,"레몬향, 민트향",34
7,17,4,a34,Lemon Mint,4,X,1,"레몬향, 민트향",34
8,72,5,a34,Lemon Mint,4,X,1,"레몬향, 민트향",34
9,82,8,a34,Lemon Mint,4,X,1,"레몬향, 민트향",34


In [20]:
data = df.pivot_table('rating', index = 'userid', columns = '차품목').fillna(0)

In [21]:
data

차품목,Bengal Spice,BerryBlossom White,Black Cherry Berry,Breakfast in Paris Black Tea,ButterScotch Blondie,Candy Cane Lane,Chai Green Tea,Chamomile Nights Herbal Tea,China Green Tips,Choco,...,"Yogi Tea, 여성용 라즈베리 잎",ZEN™-green tea,요기티 Yogi Tea 라즈베리 패션 퍼펙트 에너지 티 16티백,요기티 Yogi Tea 베드타임 티 16티백,요기티 Yogi Tea 스윗 탠저린 파지티브 에너지 티 16티백,요기티 Yogi Tea 스토맥 이즈 티 16티백,요기티 Yogi Tea 진저 티 16티백,요기티 Yogi Tea 콜드 시즌 티 16티백,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,요기티 Yogi Tea 피치 디톡스 티 16티백
userid,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
data.shape

(100, 65)

In [23]:
data = data.transpose()

In [24]:
data

userid,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
차품목,,,,,,,,,,,,,,,,,,,,,
Bengal Spice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BerryBlossom White,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Black Cherry Berry,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Breakfast in Paris Black Tea,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ButterScotch Blondie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
요기티 Yogi Tea 스토맥 이즈 티 16티백,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
요기티 Yogi Tea 진저 티 16티백,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
요기티 Yogi Tea 콜드 시즌 티 16티백,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [25]:
data.shape

(65, 100)

In [26]:
tea_sim = cosine_similarity(data, data)
print(tea_sim.shape)
#data의 user값 65개만 참고함

(65, 65)


In [27]:
tea_sim_df = pd.DataFrame(data = tea_sim, index = data.index, columns = data.index)

In [28]:
tea_sim_df.head(5)

차품목,Bengal Spice,BerryBlossom White,Black Cherry Berry,Breakfast in Paris Black Tea,ButterScotch Blondie,Candy Cane Lane,Chai Green Tea,Chamomile Nights Herbal Tea,China Green Tips,Choco,...,"Yogi Tea, 여성용 라즈베리 잎",ZEN™-green tea,요기티 Yogi Tea 라즈베리 패션 퍼펙트 에너지 티 16티백,요기티 Yogi Tea 베드타임 티 16티백,요기티 Yogi Tea 스윗 탠저린 파지티브 에너지 티 16티백,요기티 Yogi Tea 스토맥 이즈 티 16티백,요기티 Yogi Tea 진저 티 16티백,요기티 Yogi Tea 콜드 시즌 티 16티백,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,요기티 Yogi Tea 피치 디톡스 티 16티백
차품목,,,,,,,,,,,,,,,,,,,,,
Bengal Spice,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.219953,0.0,0.0,0.0
BerryBlossom White,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
Black Cherry Berry,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
Breakfast in Paris Black Tea,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
ButterScotch Blondie,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [29]:
tea_sim_df["BerryBlossom White"].sort_values(ascending=False)[1:10]

차품목
Vanilla Bean Macaron                       0.215249
Rose                                       0.195180
Cinnamon Apple Spice                       0.046004
Tumeric Orange                             0.000000
Tumeric Chai                               0.000000
Tazo Herbal Juniper Mint Honey Tea Bags    0.000000
Sweet Tangerine Positive Energy            0.000000
SleepyTime                                 0.000000
Roastaroma                                 0.000000
Name: BerryBlossom White, dtype: float64

In [30]:
tea_sim_df["Dream"].sort_values(ascending=False)[1:10]

차품목
Clear Mind Tea                             0.894427
Chocolate Mint Oolong Tea                  0.080904
Bengal Spice                               0.000000
Vanilla Bean Macaron                       0.000000
Tumeric Orange                             0.000000
Tumeric Chai                               0.000000
Tazo Herbal Juniper Mint Honey Tea Bags    0.000000
Sweet Tangerine Positive Energy            0.000000
SleepyTime                                 0.000000
Name: Dream, dtype: float64

In [31]:
tea_sim_df["ButterScotch Blondie"].sort_values(ascending=False)[1:10]

차품목
Tumeric Chai                               0.273714
Juniper Mint Honey                         0.024500
Bengal Spice                               0.000000
Rose                                       0.000000
Tumeric Orange                             0.000000
Tazo Herbal Juniper Mint Honey Tea Bags    0.000000
Sweet Tangerine Positive Energy            0.000000
SleepyTime                                 0.000000
Refresh Mint                               0.000000
Name: ButterScotch Blondie, dtype: float64

In [32]:
tea_sim_df["Tumeric Chai"].sort_values(ascending=False)[1:10]

차품목
Candy Cane Lane                    0.652092
요기티 Yogi Tea 베드타임 티 16티백           0.528195
ButterScotch Blondie               0.273714
Decaf Vanilla Chai Black Tea       0.030385
Peppermint Huckleberry             0.000000
Pomegranate Raspberry Green Tea    0.000000
PricklyPear Cactus                 0.000000
Refresh Mint                       0.000000
Vanilla Bean Macaron               0.000000
Name: Tumeric Chai, dtype: float64